In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [3]:
from toolbox import models
from django.forms.models import model_to_dict
import numpy as np
import json
from django.contrib.gis.geos import GEOSGeometry, Polygon, MultiPolygon
import pandas as pd

In [4]:
all = models.GekRetention.objects.all()

In [7]:
for gek in all:
    if gek.mnt_1_id is not None:
        n = models.GekRetentionMeasure()
        n.gek_retention = gek
        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_1_id)
        n.anz = gek.anz_1
        n.besch = gek.besch_1
        n.prio = gek.prio_1
        n.kosten = gek.kosten_1
        n.measure_number = 1
        n.save()

In [8]:
for gek in all:
    if gek.mnt_id_2 is not None:
        n = models.GekRetentionMeasure()
        n.gek_retention = gek
        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_2)
        n.anz = gek.anz_2
        n.besch = gek.besch_2
        n.prio = gek.prio_2
        n.kosten = gek.kosten_2
        n.measure_number = 2
        n.save()


        if gek.mnt_id_3 is not None:
            n = models.GekRetentionMeasure()
            n.gek_retention = gek
            n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_3)
            n.anz = gek.anz_3
            n.besch = gek.besch_3
            n.prio = gek.prio_3
            n.kosten = gek.kosten_3
            n.measure_number = 3
            n.save()

            if gek.mnt_id_4 is not None:
                n = models.GekRetentionMeasure()
                n.gek_retention = gek
                n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_4)
                n.anz = gek.anz_4
                n.besch = gek.besch_4
                n.prio = gek.prio_4
                n.kosten = gek.kosten_4
                n.measure_number = 4
                n.save()

                if gek.mnt_id_5 is not None:
                    n = models.GekRetentionMeasure()
                    n.gek_retention = gek
                    n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_5)
                    n.anz = gek.anz_5
                    n.besch = gek.besch_5
                    n.prio = gek.prio_5
                    n.kosten = gek.kosten_5
                    n.measure_number = 5
                    n.save()

                    if gek.mnt_id_6 is not None:
                        n = models.GekRetentionMeasure()
                        n.gek_retention = gek
                        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_6)
                        n.anz = gek.anz_6
                        n.besch = gek.besch_6
                        n.prio = gek.prio_6
                        n.kosten = gek.kosten_6
                        n.measure_number = 6

In [ ]:
for gek in all:
    if gek.mnt_id_6 is not None:
        n = models.GekRetentionMeasure()
        n.gek_retention = gek
        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_6)
        n.anz = gek.anz_6
        n.besch = gek.besch_6
        n.prio = gek.prio_6
        n.kosten = gek.kosten_6
        n.measure_number = 6
        n.save()for gek in all:
    if gek.mnt_id_6 is not None:
        n = models.GekRetentionMeasure()
        n.gek_retention = gek
        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_6)
        n.anz = gek.anz_6
        n.besch = gek.besch_6
        n.prio = gek.prio_6
        n.kosten = gek.kosten_6
        n.measure_number = 6
        n.save()for gek in all:
    if gek.mnt_id_6 is not None:
        n = models.GekRetentionMeasure()
        n.gek_retention = gek
        n.gek_measure = models.GEKMeasures.objects.get(pk=gek.mnt_id_6)
        n.anz = gek.anz_6
        n.besch = gek.besch_6
        n.prio = gek.prio_6
        n.kosten = gek.kosten_6
        n.measure_number = 6
        n.save()